In [2]:
import nltk
import numpy as np
import pandas as pd

1. [ ] checkout [OKSO](https://classifikators.ru/okso);
1. [ ] compare plain distances from "mathematics" to "IT", "javascript", etc and to, say, "machinist";
1. [ ] try manually specifying some skills as dependencies for some jobs and training a net which takes two word embeddings and answers whether one is a dependency for the other;
1. [ ] this isn't going to work out of the box; research what's needed to tune it
1. [ ] datasets available that can be used for adjusting the model?
    - https://www.kaggle.com/niyamatalmass/google-job-skills
    - https://www.kaggle.com/madhab/jobposts
    - https://www.reddit.com/r/dataisbeautiful/comments/25qjpz/how_many_employees_are_moving_between_companies_oc/chjvd0g/
    - https://github.com/hhru/api

In [4]:
from nltk.corpus import wordnet as wn

In [1]:
import pymongo
client = pymongo.MongoClient()
db = client['uni']
vpo1 = db.vpo1

In [8]:
programs = vpo1.aggregate(
    allowDiskUse=True,
    pipeline=[
    {
        '$match': {
            'section_name': {
                '$regex': 'Распределение численности студентов.*'
            }
        }
    }, {
        '$unwind': {
            'path': '$data', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$unwind': {
            'path': '$data.records', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$match': {
            'data.records.group': {
                '$in': [
                    1, 2, 3
                ]
            }
        }
    }, {
        '$project': {
            'name': '$data.records.name'
        }
    }, {
        '$group': {
            '_id': '$name', 
            'name': {
                '$first': '$_id.name'
            }
        }
    }, {
        '$project': {
            '_id': 1
        }
    }
])
programs = [record['_id'] for record in programs]

In [9]:
programs

['Андорра',
 'Американское Самоа',
 'Организация и безопасность движения',
 'Автомобиле- и тракторостроение',
 'Горные машины и оборудование',
 'Экономика и управление на предприятии (по отраслям)',
 'Коммерция (торговое дело)',
 'Логопедия',
 'Педагогика и психология',
 'Изобразительное искусство',
 'Искусствоведение',
 'Русский язык и литература',
 'Регионоведение',
 'Виргинские острова, США',
 'Танзания, объединенная республика',
 'Соединенное Королевство',
 'Того',
 'Таиланд',
 'Швеция',
 'Южный Судан )',
 'Словакия',
 'Сингапур',
 'Сейшелы',
 'СентВинсент и Гренадины',
 'Катар',
 'Панама',
 'Никарагуа',
 'Новая Зеландия',
 'Нидерланды',
 'Намибия',
 'Маврикий',
 'Малайзия',
 'Малави',
 'Мадагаскар',
 'Ливия',
 'Либерия',
 'Исландия',
 'Гонконг',
 'Гондурас',
 'Гайана',
 'Ангилья',
 'Гватемала',
 'Габон',
 'Эфиопия',
 'ЭльСальвадор',
 'БуркинаФасо',
 'ЦентральноАфриканская Республика',
 'КабоВерде',
 'Канада',
 'Камбоджа',
 'Мьянма',
 'Ботсвана',
 'Барбадос',
 'Антигуа и Барбуда',
